<a href="https://colab.research.google.com/github/monicafar147/regression-predict-api-template/blob/withoutdummies/testing_random_forest_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline
import math
from pprint import pprint
from sklearn import metrics
from sklearn.model_selection import train_test_split
pd.options.mode.chained_assignment = None 

# Scaling
from sklearn.preprocessing import StandardScaler

# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Machine learning models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.ensemble import BaggingRegressor

# Saving model with pickle
import pickle

sns.set(palette='pastel')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
train = pd.read_csv('https://raw.githubusercontent.com/monicafar147/regression-predict-api-template/master/Train.csv', index_col=0)
test = pd.read_csv('https://raw.githubusercontent.com/monicafar147/regression-predict-api-template/master/Test.csv', index_col=0)

In [3]:
train.head()

,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
Order No,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,12,5,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,30,2,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,9:37:56 AM,15,5,9:43:06 AM,15,5,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,10:03:53 AM,13,1,10:05:23 AM,13,1,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


In [0]:
  def _preprocess(feature_vector_df):
    feature_vector_df.columns = [col.replace("-"," ") for col in feature_vector_df.columns]
    feature_vector_df.columns = [col.replace("(Mo = 1)"," ") for col in feature_vector_df.columns]
    feature_vector_df.columns = [col.replace(" ","") for col in feature_vector_df.columns]

    # Convert time columns to datetime 
    feature_vector_df['PickupTime'] = pd.to_datetime(feature_vector_df['PickupTime']).dt.time
    #feature_vector_df['PlacementTime'] = pd.to_datetime(feature_vector_df['PlacementTime']).dt.time
    #feature_vector_df['ConfirmationTime'] = pd.to_datetime(feature_vector_df['ConfirmationTime']).dt.time
    #feature_vector_df['ArrivalatPickupTime'] = pd.to_datetime(feature_vector_df['ArrivalatPickupTime']).dt.time
    #feature_vector_df['ArrivalatDestinationTime'] = pd.to_datetime(feature_vector_df['ArrivalatDestinationTime']).dt.time

    # Converting columns into categories
    #feature_vector_df['VehicleType'] = feature_vector_df['VehicleType'].astype('category')
    #feature_vector_df['PlatformType'] = feature_vector_df['PlatformType'].astype('category')
    #feature_vector_df['PersonalorBusiness'] = feature_vector_df['PersonalorBusiness'].astype('category')

    #feature_vector_df['PlacementDay of Month'] = feature_vector_df['PlacementDayofMonth'].astype('category')
    #feature_vector_df['PlacementWeekday'] = feature_vector_df['PlacementWeekday'].astype('category')
    #feature_vector_df['ConfirmationDayofMonth'] = feature_vector_df['ConfirmationDayofMonth'].astype('category')
    #feature_vector_df['ConfirmationWeekday'] = feature_vector_df['ConfirmationWeekday'].astype('category')

    #feature_vector_df['ArrivalatPickupDayofMonth'] = feature_vector_df['ArrivalatPickupDayofMonth'].astype('category')
    #feature_vector_df['ArrivalatPickupWeekday'] = feature_vector_df['ArrivalatPickupWeekday'].astype('category')
    #feature_vector_df['ArrivalatDestinationDayofMonth'] = feature_vector_df['ArrivalatDestinationDayofMonth'].astype('category')
    #feature_vector_df['ArrivalatDestinationWeekday'] = feature_vector_df['ArrivalatDestinationWeekday'].astype('category')

    #feature_vector_df[['PickupDayofMonth', 'PickupWeekday']] = feature_vector_df[['PickupDayofMonth', 'PickupWeekday']].astype('category')
    #feature_vector_df[['PlatformType', 'PersonalorBusiness']] = feature_vector_df[['PlatformType', 'PersonalorBusiness']].astype('category')

    # Dropping rows where delivery time is less than 60s
    #feature_vector_df = feature_vector_df[feature_vector_df['TimefromPickuptoArrival'] > 60]
    
    # Selecting columns to match training data
    #feature_vector_df = feature_vector_df[['PlatformType', 'PersonalorBusiness',
    #                                   'PickupDayofMonth', 'PickupWeekday',
    #                                   'PickupTime', 'PickupLat',	'PickupLong',
    #                                   'DestinationLat',	'DestinationLong',
    #                                   'Distance(KM)', 'Temperature',
    #                                   'Precipitationinmillimeters']]

    # Selecting columns to match training data
    feature_vector_df = feature_vector_df[['PickupLat','PickupLong','DestinationLat','DestinationLong','Distance(KM)','Temperature','Precipitationinmillimeters','PickupTime']]

    # Function to assign time values into time buckets
    def assign_time_category(delivery_time):
        dts = pd.DataFrame(['00:00:00', '6:00:00', '9:00:00', '12:00:00', '15:00:00', '18:00:00'])
        dts[0] = pd.to_datetime(dts[0]).dt.time
        if delivery_time >= dts[0][0] and delivery_time < dts[0][1]:
            return 'Early Morning'
        elif delivery_time >= dts[0][1] and delivery_time < dts[0][2]:
            return 'Morning'
        elif delivery_time >= dts[0][2]  and delivery_time < dts[0][3]:
            return 'Late Morning'
        elif delivery_time >= dts[0][3] and delivery_time < dts[0][4]:
            return 'Afternoon'
        elif delivery_time >= dts[0][4] and delivery_time < dts[0][5]:
            return 'Late Afternoon'
        else:
            return 'Evening' 

    # Create new time bucket feature using the assign_time_category function
    feature_vector_df['DeliveryTimes'] = feature_vector_df['PickupTime'].apply(assign_time_category)
    feature_vector_df['DeliveryTimes'] = feature_vector_df['DeliveryTimes'].astype('category')

    # Drop 'Pickup - Time' because we have created 'Delivery Times' in its place
    #feature_vector_df.drop('PickupTime', axis=1, inplace=True)

    # Fill missing precipitation values with 0
    feature_vector_df['Precipitationinmillimeters'] = feature_vector_df['Precipitationinmillimeters'].fillna(value=0)
   
    # Impute missing temperature based on delivery time
    # Function to fill nulls with a column's mean value
    def mean(col):
      return col.fillna(col.mean())

    feature_vector_df['Temperature'] = feature_vector_df.groupby(['DeliveryTimes'])['Temperature'].transform(mean)  
    
    # Impute missing temperature based on delivery time
    feature_vector_df['Temperature'] = feature_vector_df.groupby(['DeliveryTimes'])['Temperature'].transform(mean)

    # Function to calculate a coordinate's distance from the CBD coordinate
    def distance_CBD(lat, long):
        return math.sqrt(((lat - -1.283526) ** 2) + ((long - 36.823269) ** 2))

    feature_vector_df['Distance_CBD_pickup'] = np.vectorize(distance_CBD)(feature_vector_df['PickupLat'],
                                    feature_vector_df['PickupLong'])
    feature_vector_df['Distance_CBD_dest'] = np.vectorize(distance_CBD)(feature_vector_df['DestinationLat'],
                                    feature_vector_df['DestinationLong'])
    
    # One-hot encoding for categorical data
    #feature_vector_df = pd.get_dummies(feature_vector_df,
    #                               columns=['PlatformType', 'PersonalorBusiness', 'PickupDayofMonth', 'PickupWeekday', 'DeliveryTimes'],
    #                               prefix=['platformtype', 'personalbusiness', 'dayofmonth', 'weekday', 'pickuptimes'])'''

    
    feature_vector_df = feature_vector_df[['PickupLat','PickupLong','DestinationLat','DestinationLong','Distance(KM)','Temperature','Precipitationinmillimeters','Distance_CBD_pickup','Distance_CBD_dest']]
    
    # ------------------------------------------------------------------------
    
    predict_vector = feature_vector_df

    return predict_vector

In [0]:
processed_data = _preprocess(train)


In [6]:
processed_data.head()

,PickupLat,PickupLong,DestinationLat,DestinationLong,Distance(KM),Temperature,Precipitationinmillimeters,Distance_CBD_pickup,Distance_CBD_dest
Order No,,,,,,,,,
Order_No_4211,-1.317755,36.830370,-1.300406,36.829741,4,20.400000,0.0,0.034958,0.018078
Order_No_25375,-1.351453,36.899315,-1.295004,36.814358,16,26.400000,0.0,0.101966,0.014531
Order_No_1899,-1.308284,36.843419,-1.300921,36.828195,3,24.202441,0.0,0.031922,0.018079
Order_No_9336,-1.281301,36.832396,-1.257147,36.795063,9,19.200000,0.0,0.009394,0.038619
Order_No_27883,-1.266597,36.792118,-1.295041,36.809817,9,15.400000,0.0,0.035454,0.017707


In [0]:
# train-test split
training_features = processed_data.iloc[:, :-1]
training_outputs = processed_data.iloc[:, -1]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(training_features, training_outputs, test_size=0.2)

In [9]:
# Create RandomForestRegressor object
random_forest = RandomForestRegressor(n_estimators = 7)

# Fit a random foreset model
random_forest.fit(X_train, y_train)

# Predicting test results
y_pred_forest = random_forest.predict(X_test)

print("Training MSE:", metrics.mean_squared_error(training_outputs, random_forest.predict(training_features)))
print("Validation MSE:", metrics.mean_squared_error(y_test, y_pred_forest))

print("\nTraining RMSE:", np.sqrt(metrics.mean_squared_error(training_outputs, random_forest.predict(training_features))))
print("Validation RMSE:", np.sqrt(metrics.mean_squared_error(y_test, y_pred_forest)))

Training MSE: 1.576559950206918e-06
Validation MSE: 4.594496981713097e-06

Training RMSE: 0.001255611385026003
Validation RMSE: 0.0021434777772846394


In [0]:
#model_load_path = "rfm.pkl"
#with open(model_load_path,'rb') as file:
#    unpickled_model = pickle.load(file)

In [0]:
#y_pred = unpickled_model.predict(processed_data)